In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [22]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import multiprocessing
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
dataset = load_dataset("ashraq/fashion-product-images-small")

In [24]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName', 'image'],
        num_rows: 44072
    })
})


In [135]:
data = pd.DataFrame(dataset['train'])

In [136]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,<PIL.Image.Image image mode=L size=60x80 at 0x...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,<PIL.Image.Image image mode=RGB size=60x80 at ...


In [139]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [140]:
# Data cleaning
data['year'] = data['year'].apply(lambda x : str(int(x)))

In [141]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,<PIL.Image.Image image mode=L size=60x80 at 0x...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,<PIL.Image.Image image mode=RGB size=60x80 at ...


In [142]:
# data reduction remoiving unsual columns
data = data.drop(columns=['image'])

In [143]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt


In [144]:
# checking for null values
data.isna().sum()

id                    0
gender                0
masterCategory        0
subCategory           0
articleType           0
baseColour            0
season                0
year                  0
usage                 0
productDisplayName    0
dtype: int64

In [147]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt


In [148]:
# Data extraction - extraction Brand name form Product Display Name
data['BrandName'] = data['productDisplayName'].apply(lambda x: "".join(x.split()[0:min(2,len(x.split()))]))

In [149]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,BrandName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,TurtleCheck
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,PeterEngland
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,TitanWomen
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,ManchesterUnited
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,PumaMen


In [150]:
# Data reduction - removing unwanted columns
data.drop(columns=['productDisplayName'],inplace=True)

In [151]:

# Data preprocessing to make it fit for vectorization
for col in data:
  if(col=='id'): continue
  data[col] = data[col].apply(lambda x : [x.replace(" ","")])

In [152]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,BrandName
0,15970,[Men],[Apparel],[Topwear],[Shirts],[NavyBlue],[Fall],[2011],[Casual],[TurtleCheck]
1,39386,[Men],[Apparel],[Bottomwear],[Jeans],[Blue],[Summer],[2012],[Casual],[PeterEngland]
2,59263,[Women],[Accessories],[Watches],[Watches],[Silver],[Winter],[2016],[Casual],[TitanWomen]
3,21379,[Men],[Apparel],[Bottomwear],[TrackPants],[Black],[Fall],[2011],[Casual],[ManchesterUnited]
4,53759,[Men],[Apparel],[Topwear],[Tshirts],[Grey],[Summer],[2012],[Casual],[PumaMen]


In [153]:
data['Tag'] = data['gender']
for col in data:
  if(col=='id' or col=='gender'): continue
  data['Tag'] += data[col]

In [154]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,BrandName,Tag
0,15970,[Men],[Apparel],[Topwear],[Shirts],[NavyBlue],[Fall],[2011],[Casual],[TurtleCheck],"[Men, Apparel, Topwear, Shirts, NavyBlue, Fall..."
1,39386,[Men],[Apparel],[Bottomwear],[Jeans],[Blue],[Summer],[2012],[Casual],[PeterEngland],"[Men, Apparel, Bottomwear, Jeans, Blue, Summer..."
2,59263,[Women],[Accessories],[Watches],[Watches],[Silver],[Winter],[2016],[Casual],[TitanWomen],"[Women, Accessories, Watches, Watches, Silver,..."
3,21379,[Men],[Apparel],[Bottomwear],[TrackPants],[Black],[Fall],[2011],[Casual],[ManchesterUnited],"[Men, Apparel, Bottomwear, TrackPants, Black, ..."
4,53759,[Men],[Apparel],[Topwear],[Tshirts],[Grey],[Summer],[2012],[Casual],[PumaMen],"[Men, Apparel, Topwear, Tshirts, Grey, Summer,..."


In [156]:
data['Tag'] = data['Tag'].apply(lambda x: " ".join(x).lower())

In [157]:
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,BrandName,Tag
0,15970,[Men],[Apparel],[Topwear],[Shirts],[NavyBlue],[Fall],[2011],[Casual],[TurtleCheck],men apparel topwear shirts navyblue fall 2011 ...
1,39386,[Men],[Apparel],[Bottomwear],[Jeans],[Blue],[Summer],[2012],[Casual],[PeterEngland],men apparel bottomwear jeans blue summer 2012 ...
2,59263,[Women],[Accessories],[Watches],[Watches],[Silver],[Winter],[2016],[Casual],[TitanWomen],women accessories watches watches silver winte...
3,21379,[Men],[Apparel],[Bottomwear],[TrackPants],[Black],[Fall],[2011],[Casual],[ManchesterUnited],men apparel bottomwear trackpants black fall 2...
4,53759,[Men],[Apparel],[Topwear],[Tshirts],[Grey],[Summer],[2012],[Casual],[PumaMen],men apparel topwear tshirts grey summer 2012 c...


In [158]:
processed_data = data[['id','Tag']]

In [159]:
processed_data.head()

,id,Tag
0,15970,men apparel topwear shirts navyblue fall 2011 ...
1,39386,men apparel bottomwear jeans blue summer 2012 ...
2,59263,women accessories watches watches silver winte...
3,21379,men apparel bottomwear trackpants black fall 2...
4,53759,men apparel topwear tshirts grey summer 2012 c...


In [160]:
tmp_processed_data = processed_data.copy()



processed_data.head()

,id,Tag
0,15970,men apparel topwear shirts navyblue fall 2011 ...
1,39386,men apparel bottomwear jeans blue summer 2012 ...
2,59263,women accessories watches watches silver winte...
3,21379,men apparel bottomwear trackpants black fall 2...
4,53759,men apparel topwear tshirts grey summer 2012 c...


In [161]:
print(len(processed_data))

44072


In [162]:
tag_ind_map = {}
index_list = processed_data.index.tolist()
for i in index_list:
  tag_ind_map[processed_data['Tag'][i]] = i

# BOW

In [163]:
vectorizer = CountVectorizer()

X_spa = vectorizer.fit_transform(processed_data['Tag'])
vocab = vectorizer.get_feature_names_out()

X = X_spa.toarray()

print(X.shape)

(44072, 1958)


In [164]:
print(vocab)

['109fblue' '109fred' '109fwomen' ... 'yorkmen' 'yorkwomen' 'yvessaint']


In [165]:
adj_lst = [dict() for i in range(len(processed_data)) ]
cnt = 0

for sen_id, wrd_id in zip(*X_spa.nonzero()):
    fre = X_spa[sen_id, wrd_id]
    adj_lst[sen_id][wrd_id]=int(fre)


In [166]:
processed_data['BOW'] = adj_lst

<ipython-input-166-04958e981016>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['BOW'] = adj_lst


In [167]:
processed_data.head()

,id,Tag,BOW
0,15970,men apparel topwear shirts navyblue fall 2011 ...,"{1148: 2, 112: 2, 1825: 2, 1648: 2, 1215: 2, 7..."
1,39386,men apparel bottomwear jeans blue summer 2012 ...,"{1148: 2, 112: 2, 296: 2, 223: 2, 939: 2, 216:..."
2,59263,women accessories watches watches silver winte...,"{296: 2, 1922: 2, 26: 2, 1901: 4, 1657: 2, 192..."
3,21379,men apparel bottomwear trackpants black fall 2...,"{1148: 2, 112: 2, 716: 2, 9: 2, 296: 2, 223: 2..."
4,53759,men apparel topwear tshirts grey summer 2012 c...,"{1148: 2, 112: 2, 1825: 2, 296: 2, 1755: 2, 10..."


In [168]:
print(len(processed_data['BOW'][0]))

9


# Weighted Word2Vec using TFIDF

In [169]:
# Word2Vec + TFIDF

tfidf_vectorizer = TfidfVectorizer(tokenizer = lambda x : x.split())

tfidf_matrix = tfidf_vectorizer.fit_transform(processed_data['Tag'])

feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_matrix = tfidf_matrix.toarray()

print(tfidf_matrix)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [170]:
ind_map = {}

for index,val in enumerate(feature_names):
    ind_map[val] = index

In [171]:
num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 2


In [172]:

# tokenized_sentences = [word_tokenize(sentence) for sentence in processed_data['Tag']]
# print(tokenized_sentences)

tokenized_sentences = [sentence.split() for sentence in processed_data['Tag']]
model = Word2Vec(sentences=tokenized_sentences, vector_size=159, window=5, min_count=1, workers=num_cores-1)



In [173]:
word_vector = model.wv['casiog-shock']

similar_words = model.wv.most_similar('casiog-shock')

In [174]:
print(word_vector)

[ 0.02252007  0.17848064  0.30650243 -0.0047119  -0.07990095  0.26934496
  0.20875071 -0.09626729 -0.32250577 -0.07011883  0.06669473 -0.2206737
 -0.05657922  0.36111614 -0.03583889 -0.31325755  0.01873717 -0.16781273
  0.02343929  0.1056317   0.17914629 -0.05050506 -0.31162518  0.08698863
 -0.05314419 -0.10134269 -0.19154622  0.2554021   0.01189535  0.03848442
 -0.30248573  0.08388971  0.11610574 -0.09855466  0.13906194  0.14358139
 -0.05167289 -0.24330182 -0.00529674 -0.24140097 -0.21581158  0.06046322
  0.16951758 -0.26856467 -0.28876927 -0.06960656 -0.0720371   0.09335005
 -0.15255153  0.1030021   0.03762895 -0.04182747 -0.05180977  0.12191044
  0.03346921  0.07149709  0.1095458  -0.12939154 -0.2121246   0.22950919
 -0.02578628 -0.26986519 -0.07211552 -0.17948471  0.15566361 -0.10613696
 -0.17215823  0.32872587  0.03284064  0.18709758  0.19957612 -0.23885652
 -0.19678259  0.17176555  0.03687622 -0.06826184 -0.40714186 -0.04280803
 -0.06043229 -0.02856933  0.01577323  0.1326102  -0.

In [175]:
print(similar_words)

[('nauticamen', 0.9320828914642334), ('giordanomen', 0.9150955080986023), ('q&qmen', 0.9130545258522034), ('casioyouth', 0.9130365252494812), ('casioedifice', 0.8974304795265198), ('carreramen', 0.8884420990943909), ('titanmen', 0.8873603343963623), ('maximamen', 0.8833813071250916), ('timexmen', 0.882756233215332), ('citizenmen', 0.8798049092292786)]


In [176]:
def convert(x,i):
    lst = np.zeros(159)
    for wrd in x.split():
        wrd2vec_vector = list(model.wv[wrd])
        tfidf_score = tfidf_matrix[i][ind_map[wrd]]
        wrd2vec_vector = np.array(wrd2vec_vector)

        wrd2vec_vector *= tfidf_score
        lst += wrd2vec_vector
    lst /= len(x.split())
    return lst


In [177]:
lst = convert(processed_data['Tag'][0],0)

In [178]:
processed_data['wrd2vec'] = [convert(j,i) for i,j in enumerate(processed_data['Tag'])]

<ipython-input-178-0ad96eeb07ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['wrd2vec'] = [convert(j,i) for i,j in enumerate(processed_data['Tag'])]


In [179]:
processed_data.head()

,id,Tag,BOW,wrd2vec
0,15970,men apparel topwear shirts navyblue fall 2011 ...,"{1148: 2, 112: 2, 1825: 2, 1648: 2, 1215: 2, 7...","[0.05704307701024744, -0.04891449109547668, 0...."
1,39386,men apparel bottomwear jeans blue summer 2012 ...,"{1148: 2, 112: 2, 296: 2, 223: 2, 939: 2, 216:...","[-0.0538736575593551, 0.08224543266826206, 0.1..."
2,59263,women accessories watches watches silver winte...,"{296: 2, 1922: 2, 26: 2, 1901: 4, 1657: 2, 192...","[-0.11195660041024287, 0.1914643837759892, 0.4..."
3,21379,men apparel bottomwear trackpants black fall 2...,"{1148: 2, 112: 2, 716: 2, 9: 2, 296: 2, 223: 2...","[0.01708908234205511, 0.04222696812616454, 0.1..."
4,53759,men apparel topwear tshirts grey summer 2012 c...,"{1148: 2, 112: 2, 1825: 2, 296: 2, 1755: 2, 10...","[-0.01710337053777443, 0.038468338549137115, 0..."


In [180]:
processed_data = processed_data.reset_index(drop=True)

In [181]:
processed_data.head()

,id,Tag,BOW,wrd2vec
0,15970,men apparel topwear shirts navyblue fall 2011 ...,"{1148: 2, 112: 2, 1825: 2, 1648: 2, 1215: 2, 7...","[0.05704307701024744, -0.04891449109547668, 0...."
1,39386,men apparel bottomwear jeans blue summer 2012 ...,"{1148: 2, 112: 2, 296: 2, 223: 2, 939: 2, 216:...","[-0.0538736575593551, 0.08224543266826206, 0.1..."
2,59263,women accessories watches watches silver winte...,"{296: 2, 1922: 2, 26: 2, 1901: 4, 1657: 2, 192...","[-0.11195660041024287, 0.1914643837759892, 0.4..."
3,21379,men apparel bottomwear trackpants black fall 2...,"{1148: 2, 112: 2, 716: 2, 9: 2, 296: 2, 223: 2...","[0.01708908234205511, 0.04222696812616454, 0.1..."
4,53759,men apparel topwear tshirts grey summer 2012 c...,"{1148: 2, 112: 2, 1825: 2, 296: 2, 1755: 2, 10...","[-0.01710337053777443, 0.038468338549137115, 0..."


In [183]:
bow_np = np.array([data for data in processed_data['BOW']])

In [184]:
print(bow_np[0:10])

[{1148: 2, 112: 2, 1825: 2, 1648: 2, 1215: 2, 716: 2, 9: 2, 296: 2, 1849: 2}
 {1148: 2, 112: 2, 296: 2, 223: 2, 939: 2, 216: 2, 1755: 2, 10: 2, 1306: 2}
 {296: 2, 1922: 2, 26: 2, 1901: 4, 1657: 2, 1921: 2, 14: 2, 1811: 2}
 {1148: 2, 112: 2, 716: 2, 9: 2, 296: 2, 223: 2, 1833: 2, 213: 2, 1128: 2}
 {1148: 2, 112: 2, 1825: 2, 296: 2, 1755: 2, 10: 2, 1846: 2, 859: 2, 1402: 2}
 {1148: 2, 112: 2, 1825: 2, 9: 2, 296: 2, 1755: 2, 1846: 2, 859: 2, 922: 2}
 {1148: 2, 112: 2, 1825: 2, 1648: 2, 1755: 2, 10: 2, 857: 2, 672: 2, 693: 2}
 {112: 2, 1825: 2, 1648: 2, 296: 2, 1755: 2, 10: 2, 1922: 2, 1417: 2, 938: 2}
 {1148: 2, 1215: 2, 296: 2, 1755: 2, 10: 2, 26: 2, 1402: 2, 1678: 4}
 {1148: 2, 296: 2, 26: 2, 1901: 4, 1921: 2, 14: 2, 213: 2, 1660: 2}]


In [186]:
print(processed_data['id'][0])

15970


In [187]:
print(data['id'].shape)

(44072,)


In [188]:
cosin_sim_mat=np.zeros(shape=(44073,44073))

In [189]:
def calc_cosine(x,y):

    mod_a=np.array([i for i in x.values()])
    mod_b=np.array([i for i in y.values()])
    mod_a=(np.sum(mod_a**2))**0.5
    mod_b=(np.sum(mod_b**2))**0.5

    mod_ab=0
    for i in y:
        if i in x:
            mod_ab+=(y[i]*x[i])
    if(mod_a==0 or mod_b==0):
        print("heheh waste bhes waste")
        return 0
    return (mod_ab/(mod_a*mod_b))





In [202]:
ind_map2 = {}

for ind,id in enumerate(processed_data['id']):
  ind_map2[id] = ind

In [203]:
print(ind_map2[15970])

0


In [206]:
print(processed_data['wrd2vec'][0])
print(processed_data['BOW'][0])

[ 0.05704308 -0.04891449  0.17894928  0.1096357   0.22445593  0.07808609
 -0.117513   -0.142553    0.01314291 -0.08896257  0.09243225  0.04584767
  0.03330345  0.11837874  0.16231564  0.0557248  -0.1153877  -0.03554353
 -0.19395862 -0.0687382  -0.02302028 -0.05583838  0.02557406  0.01734793
  0.13857702 -0.14025619 -0.1112588  -0.09502243  0.05479578  0.15590035
 -0.0354153   0.02849023  0.04110763 -0.18016372 -0.03119184 -0.21623941
  0.03857409  0.06343798 -0.12103347  0.04542022 -0.08762532  0.00597494
 -0.18502093  0.10812893 -0.01657847 -0.0627017   0.13922519 -0.01300611
  0.01418274  0.06246953  0.05377655 -0.00070272  0.09283451  0.05288547
  0.00887691 -0.14577782  0.07355216  0.10943286 -0.00395828  0.02262639
  0.11866948  0.13692895 -0.09624889 -0.04309178 -0.04288486 -0.08981184
  0.0098754  -0.09835242  0.18914035  0.0120441  -0.13585118  0.0905579
  0.00483377  0.05166845  0.12785549  0.05151835  0.07603139  0.23987104
 -0.04216064 -0.01451883  0.0202951   0.11417106  0.

In [208]:
cloth_id = int(input("Enter clothe ID : "))
# main_tup=dic_id_tup[cloth_id]
# main_id=dic_tup_id[main_tup]
# cloth_id=main_id
cloth_ind = ind_map2[cloth_id]
cloth_tag = processed_data['Tag'][cloth_ind]
new_ind = tag_ind_map[cloth_tag]
print(processed_data['Tag'][new_ind])

print("Recommended Clothes : ")
print("-"*65)

y = processed_data['BOW'][new_ind]
lst = []

for i in range(len(processed_data)):
  x = processed_data['BOW'][i]
  sim = calc_cosine(x,y)

  lst.append([sim,i])

lst.sort(reverse=True)
final_list=[]
list_set=[]
top_5 = []

for i in lst :
  if(processed_data['Tag'][i[1]] not in list_set):
    list_set.append(processed_data['Tag'][i[1]])
    top_5.append(i)
  if(len(top_5)>=5):break



for cloth in top_5:
  print(processed_data['id'][cloth[1]],processed_data['Tag'][cloth[1]])

Enter clothe ID : 47359
women accessories bags handbags brown summer 2012 casual baggitwomen women accessories bags handbags brown summer 2012 casual baggitwomen
Recommended Clothes : 
-----------------------------------------------------------------
38441 women accessories bags handbags brown summer 2012 casual baggitwomen women accessories bags handbags brown summer 2012 casual baggitwomen
33633 women accessories bags handbags brown summer 2012 casual linoperros women accessories bags handbags brown summer 2012 casual linoperros
38446 women accessories bags handbags beige summer 2012 casual baggitwomen women accessories bags handbags beige summer 2012 casual baggitwomen
33406 women accessories bags handbags olive summer 2012 casual baggitwomen women accessories bags handbags olive summer 2012 casual baggitwomen
33430 women accessories bags handbags blue summer 2012 casual baggitwomen women accessories bags handbags blue summer 2012 casual baggitwomen


In [209]:
cloth_id = int(input("Enter clothe ID : "))

# main_tup=dic_id_tup[cloth_id]
# main_id=dic_tup_id[main_tup]
# cloth_id=main_id

cloth_ind = ind_map2[cloth_id]
cloth_tag = tmp_processed_data['Tag'][cloth_ind]
new_ind = tag_ind_map[cloth_tag]
print(processed_data['Tag'][new_ind])

print("Recommended Clothes : ")
print("-"*65)

y = processed_data['wrd2vec'][new_ind].reshape(1, -1)
lst = []

for i in range(len(processed_data)):
  x = processed_data['wrd2vec'][i].reshape(1, -1)
  sim = cosine_similarity(x,y)

  lst.append([sim,i])

lst.sort(reverse=True)
top_5 = []
final_list=[]
list_set=[]
top_5 = []

for i in lst :
  if(processed_data['Tag'][i[1]] not in list_set):
    list_set.append(processed_data['Tag'][i[1]])
    top_5.append(i)
  if(len(top_5)>=5):break

for cloth in top_5:
  print(processed_data['id'][cloth[1]],processed_data['Tag'][cloth[1]])

Enter clothe ID : 47359
women accessories bags handbags brown summer 2012 casual baggitwomen women accessories bags handbags brown summer 2012 casual baggitwomen
Recommended Clothes : 
-----------------------------------------------------------------
38441 women accessories bags handbags brown summer 2012 casual baggitwomen women accessories bags handbags brown summer 2012 casual baggitwomen
33455 women accessories bags handbags brown fall 2012 casual baggitwomen women accessories bags handbags brown fall 2012 casual baggitwomen
38436 women accessories bags handbags black summer 2012 casual baggitwomen women accessories bags handbags black summer 2012 casual baggitwomen
47343 women accessories bags handbags tan summer 2012 casual baggitwomen women accessories bags handbags tan summer 2012 casual baggitwomen
28448 women accessories bags handbags brown summer 2012 casual kiarawomen women accessories bags handbags brown summer 2012 casual kiarawomen
